# Preamble

In [ ]:
!pip3 install git+https://github.com/aims50toolbox/aitoolbox.git

In [2]:
import aitoolbox.context as aictx

context = aictx.Context.get()
sources = context.get_sources()
sources.set("prompt","What is the ultimate answer?")

context.set_env("MILVUS_HOST", "localhost")
context.set_env("MILVUS_PORT", "19530")

# Setup

In [3]:
from pymilvus import connections, db, Collection
connections.connect(
  alias="default",
  user='',
  password='',
  host=context.get_env("MILVUS_HOST"),
  port=context.get_env("MILVUS_PORT")
)

db.using_database("embeddings")
chunks = Collection("chunks")

In [4]:
chunks.load()

In [5]:
from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/storage/projects/AIMS5.0/stuff/tools/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/storage/projects/AIMS5.0/stuff/tools/.venv/lib/python3.8/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


# Service

In [7]:
PROMPT=aictx.Context.get().get_sources().get("prompt")
print(PROMPT)

What is the ultimate answer?


In [8]:
embedding = embed_model.encode(PROMPT)

In [9]:
search_params = {
    "metric_type": "COSINE",
    "limit": 5,
    "params": {"nprobe": 10},
}

try:
    result = chunks.search([list(embedding)], "chunk_embedding", search_params, limit=3, output_fields=["chunk"])
except Exception as e:
    print(e)

In [10]:
result[0].distances

[0.5891436338424683, 0.46394670009613037, 0.4566752314567566]

In [11]:
query = "\n".join([ hit.entity.get('chunk') for hit in result[0]])
print(query)

"That's a pity," said Arthur. "I'd like to hear what  he  had  to say.  Presumably  he would know what the Ultimate Question to the Ultimate Answer is. It's always bothered me that we  never  found out."

"Think of a number," said the computer, " any number."
For seven and a half million years,  Deep  Thought  computed  and calculated,  and in the end announced that the answer was in fact Forty-two - and so another, even bigger, computer had to be built to find out what the actual question was.
"The Universe!" said Lunkwill.

"Everything!" they said in chorus.

Deep Thought paused for a moment's reflection.

"Tricky," he said finally.

"But can you do it?"

Again, a significant pause.

"Yes," said Deep Thought, "I can do it."

"There is an answer?" said Fook with breathless excitement."

"A simple answer?" added Lunkwill.

"Yes," said Deep Thought. "Life, the  Universe,  and  Everything. There  is  an  answer.  But," he added, "I'll have to think about it."


In [12]:
aictx.Context.get().get_destinations().set("resp",[ hit.entity.get('chunk') for hit in result[0]])